#Libraries and Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
#load the data
data = yfinance.download (tickers = "TATAPOWER.NS",start="2010-03-06",
                               interval = "1d", group_by = 'ticker', auto_adjust = True)

data.tail()
#data = pd.read_csv("DHS_Daily_Report_2020.csv")
#data.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume
Date,,,,,
2023-07-05,220.949997,223.750000,220.550003,221.699997,5879342
2023-07-06,225.000000,230.300003,223.550003,229.600006,30885511
2023-07-07,229.800003,232.350006,226.100006,228.000000,16592198
2023-07-10,229.250000,229.250000,224.649994,225.000000,7143637
2023-07-12,226.250000,227.250000,225.199997,226.300003,7362493


In [5]:
df = pd.DataFrame()

df['ds'] = pd.to_datetime(data.index.values,
                          format = "%Y-%m-%d")
df['y'] = data["Close"]
# df['pe'] = nifty_pe['P/E'].values

In [6]:
df

,ds,y
0,2010-03-08,NaN
1,2010-03-09,NaN
2,2010-03-10,NaN
3,2010-03-11,NaN
4,2010-03-12,NaN
...,...,...
3291,2023-07-05,NaN
3292,2023-07-06,NaN
3293,2023-07-07,NaN
3294,2023-07-10,NaN


#Data preparation

In [5]:
#transform the date variable
data.Date = pd.to_datetime(data.Date, format = "%m/%d/%Y")
data.Date

AttributeError: 'DataFrame' object has no attribute 'Date'

In [4]:
#final dataset
#df = data.loc[data.Date <= "2020-12-01",
 #             ["Date", "Total Individuals in Shelter",
  #             "Easter", "Thanksgiving", "Christmas"]]
#df.head()              

,Date,Total Individuals in Shelter,Easter,Thanksgiving,Christmas
0,2014-01-01,50689,0,0,0
1,2014-01-02,50797,0,0,0
2,2014-01-03,50954,0,0,0
3,2014-01-04,50957,0,0,0
4,2014-01-05,50957,0,0,0


In [5]:
#renaming variables
df = df.rename(columns = {'Total Individuals in Shelter': 'y',
                          'Date': 'ds'})
df.head(1)

,ds,y,Easter,Thanksgiving,Christmas
0,2014-01-01,50689,0,0,0


#Holidays

In [6]:
#Easter
easter_dates = df[df.Easter == 1].ds
easter = pd.DataFrame({'holiday': 'easter',
                       'ds': pd.to_datetime(easter_dates),
                       'lower_window': -5,
                       'upper_window': 2})
easter.head(2)

,holiday,ds,lower_window,upper_window
109,easter,2014-04-20,-5,2
459,easter,2015-04-05,-5,2


In [7]:
#Thanksgiving
thanksgiving_dates = df[df.Thanksgiving == 1].ds
thanksgiving = pd.DataFrame({'holiday': 'thanksgiving',
                       'ds': pd.to_datetime(thanksgiving_dates),
                       'lower_window': -5,
                       'upper_window': 2})
thanksgiving.head(2)

,holiday,ds,lower_window,upper_window
330,thanksgiving,2014-11-27,-5,2
694,thanksgiving,2015-11-26,-5,2


In [8]:
#combining holidays
holidays = pd.concat([easter, thanksgiving])
holidays

,holiday,ds,lower_window,upper_window
109,easter,2014-04-20,-5,2
459,easter,2015-04-05,-5,2
816,easter,2016-03-27,-5,2
1201,easter,2017-04-16,-5,2
1551,easter,2018-04-01,-5,2
1936,easter,2019-04-21,-5,2
2293,easter,2020-04-12,-5,2
330,thanksgiving,2014-11-27,-5,2
694,thanksgiving,2015-11-26,-5,2
1058,thanksgiving,2016-11-24,-5,2


#Prophet

In [9]:
m = Prophet(holidays = holidays,
            seasonality_mode = 'multiplicative',
            seasonality_prior_scale = 10,
            changepoint_prior_scale = 0.05,
            holidays_prior_scale = 10)
m.add_regressor('Christmas')
m.fit(df)

16:31:39 - cmdstanpy - INFO - Chain [1] start processing
16:31:42 - cmdstanpy - INFO - Chain [1] done processing


In [11]:
#CV
from prophet.diagnostics import cross_validation
df_cv = cross_validation(m,
                         horizon = '31 days',
                         initial = '2400 days',
                         parallel = "processes")

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
16:32:06 - cmdstanpy - INFO - Chain [1] start processing
16:32:06 - cmdstanpy - INFO - Chain [1] start processing
16:32:06 - cmdstanpy - INFO - Chain [1] start processing
16:32:06 - cmdstanpy - INFO - Chain [1] start processing
16:32:13 - cmdstanpy - INFO - Chain [1] done processing
16:32:14 - cmdstanpy - INFO - Chain [1] done processing
16:32:14 - cmdstanpy - INFO - Chain [1] done processing
16:32:14 - cmdstanpy - INFO - Chain [1] done processing
16:32:15 - cmdstanpy - INFO - Chain [1] start processing
16:32:15 - cmdstanpy - INFO - Chain [1] start processing
16:32:15 - cmdstanpy - INFO - Chain [1] start processing
16:32:20 - cmdstanpy - INFO - Chain [1] done processing
16:32:21 - cmdstanpy - INFO - Chain [1] done processing
16:32:21 - cmdstanpy - INFO - Cha

In [12]:
#assess model
from sklearn.metrics import mean_absolute_error
mean_absolute_error(df_cv.yhat, df_cv.y)

1996.4039732500853

#Hyperparameter tuning

In [13]:
#PArameters
from sklearn.model_selection import ParameterGrid
param_grid = {'seasonality_prior_scale': [10, 20],
            'changepoint_prior_scale': [0.05, 0.1],
            'holidays_prior_scale': [10, 20]}
grid = ParameterGrid(param_grid)

In [14]:
#Tuning  
mae = []
for params in grid:
  #build model
  m = Prophet(holidays = holidays,
            seasonality_mode = 'multiplicative',
            seasonality_prior_scale = params['seasonality_prior_scale'],
            changepoint_prior_scale = params['changepoint_prior_scale'],
            holidays_prior_scale = params['holidays_prior_scale'])
  m.add_regressor('Christmas')
  m.fit(df)

  #CV
  df_cv = cross_validation(m,
                         horizon = '31 days',
                         initial = '2400 days',
                         parallel = "processes")

  #Gather the results
  error = mean_absolute_error(df_cv.yhat, df_cv.y)
  mae.append(error)

16:32:46 - cmdstanpy - INFO - Chain [1] start processing
16:32:48 - cmdstanpy - INFO - Chain [1] done processing
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
16:32:51 - cmdstanpy - INFO - Chain [1] start processing
16:32:51 - cmdstanpy - INFO - Chain [1] start processing
16:32:51 - cmdstanpy - INFO - Chain [1] start processing
16:32:51 - cmdstanpy - INFO - Chain [1] start processing
16:32:58 - cmdstanpy - INFO - Chain [1] done processing
16:32:59 - cmdstanpy - INFO - Chain [1] done processing
16:32:59 - cmdstanpy - INFO - Chain [1] done processing
16:32:59 - cmdstanpy - INFO - Chain [1] done processing
16:32:59 - cmdstanpy - INFO - Chain [1] start processing
16:33:00 - cmdstanpy - INFO - Chain [1] start processing
16:33:00 - cmdstanpy - INFO - Chain [1] start processing
16:33:05 - cmdstanpy - INFO - Ch

16:35:16 - cmdstanpy - INFO - Chain [1] start processing
16:35:16 - cmdstanpy - INFO - Chain [1] start processing
16:35:16 - cmdstanpy - INFO - Chain [1] start processing
16:35:17 - cmdstanpy - INFO - Chain [1] start processing
16:35:23 - cmdstanpy - INFO - Chain [1] done processing
16:35:24 - cmdstanpy - INFO - Chain [1] start processing
16:35:24 - cmdstanpy - INFO - Chain [1] done processing
16:35:25 - cmdstanpy - INFO - Chain [1] done processing
16:35:25 - cmdstanpy - INFO - Chain [1] done processing
16:35:25 - cmdstanpy - INFO - Chain [1] start processing
16:35:26 - cmdstanpy - INFO - Chain [1] start processing
16:35:30 - cmdstanpy - INFO - Chain [1] done processing
16:35:30 - cmdstanpy - INFO - Chain [1] done processing
16:35:31 - cmdstanpy - INFO - Chain [1] done processing


In [15]:
#gather best parameters
best_params = grid[np.argmin(mae)]
print(best_params)

{'seasonality_prior_scale': 20, 'holidays_prior_scale': 20, 'changepoint_prior_scale': 0.1}
